In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
from lightgbm import LGBMClassifier
import os
import eli5
from eli5.sklearn import PermutationImportance
from catboost import CatBoostClassifier

In [2]:
train = pd.read_csv("C:/Users/Aditya/Desktop/HR Analytics/train.csv")
test = pd.read_csv("C:/Users/Aditya/Desktop/HR Analytics/test.csv")
sample = pd.read_csv("C:/Users/Aditya/Desktop/HR Analytics/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
duplicate_train = train[train.duplicated()]

In [ ]:
duplicate_train

In [ ]:
train.info()

In [ ]:
sns.distplot(train['age'])

In [ ]:
object_col = []

for column in train.columns:
    if train[column].dtype == object and len(train[column].unique()) <=30:
        object_col.append (column)
        print (f"{column} : {train[column].unique()}")
        print (train[column].value_counts())
        print ("================")

In [ ]:
train.isnull().sum()

In [ ]:
train['education'].value_counts()

In [3]:
train['education'] = train['education'].fillna("Bachelor's")

In [ ]:
train['previous_year_rating'].value_counts()

In [ ]:
sns.distplot(train['previous_year_rating'])

In [4]:
train['previous_year_rating'] = train['previous_year_rating'].fillna(3.0)

In [ ]:
train.head()

In [ ]:
test.isnull().sum()

In [5]:
test['education'] = test['education'].fillna("Bachelor's")
test['previous_year_rating'] = test['previous_year_rating'].fillna(3.0)

In [ ]:
test['previous_year_rating'].value_counts()

In [ ]:
sns.barplot(train['is_promoted'],train['is_promoted'].value_counts())

In [ ]:
train['is_promoted'].value_counts()

## Performing LGBM:

In [ ]:
#train=train.drop_duplicates(subset=[ele for ele in list(train.columns) if ele not in ['employee_id']])

In [ ]:
#train.shape

In [6]:
combine_set=pd.concat([train,test], ignore_index=True)

In [ ]:
combine_set.shape

In [ ]:
combine_set.head()

In [7]:
le=LabelEncoder()

In [8]:
combine_set['gender']=le.fit_transform(combine_set['gender'])
combine_set['department']=le.fit_transform(combine_set['department'])
#combine_set['recruitment_channel']=le.fit_transform(combine_set['recruitment_channel'])

In [ ]:
combine_set['education'].value_counts()

In [9]:
combine_set['education']=combine_set['education'].map({'Below Secondary':1,"Bachelor's":2,"Master's & above":3})

In [ ]:
combine_set.tail()

In [10]:
combine_set[['region1','region']] =  combine_set['region'].str.split("_", expand = True)

In [36]:
combine_set.drop(['region1'], axis = 1, inplace = True)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(combine_set.corr(),annot=True,fmt=".2f")

In [11]:
combine_set['previous_year_rating'] = combine_set['previous_year_rating'].astype('int')

In [12]:
combine_set['Total_Score'] = combine_set['no_of_trainings'] * combine_set['avg_training_score']

In [13]:
combine_set['departmentwise_rating_sum']=combine_set.groupby(['department'])['previous_year_rating'].transform('sum')
#combine_set['departmentwise_rating_SD']=combine_set.groupby(['department'])['previous_year_rating'].transform('std')
#combine_set['departmentwise_rating_uni']=combine_set.groupby(['department'])['previous_year_rating'].transform('nunique')


combine_set['recruitment_rating_sum']=combine_set.groupby(['recruitment_channel'])['previous_year_rating'].transform('sum')
#combine_set['recruitment_rating_SD']=combine_set.groupby(['recruitment_channel'])['previous_year_rating'].transform('std')
#combine_set['recruitment_rating_uni']=combine_set.groupby(['recruitment_channel'])['previous_year_rating'].transform('nunique')

In [ ]:
#combine_set['departmentwise_rating_mod']=combine_set.groupby(['department'])['previous_year_rating'].agg(lambda x:x.value_counts().index[0])

In [ ]:
combine_set.head()

In [14]:
combine_set['education_rating_sum']=combine_set.groupby(['education'])['previous_year_rating'].transform('sum')
#combine_set['education_rating_SD']=combine_set.groupby(['education'])['previous_year_rating'].transform('std')
#combine_set['education_rating_uni']=combine_set.groupby(['education'])['previous_year_rating'].transform('nunique')

In [15]:
combine_set['awards_won_rating_sum']=combine_set.groupby(['awards_won?'])['previous_year_rating'].transform('sum')
#combine_set['service_length_rating_sd']=combine_set.groupby(['length_of_service'])['previous_year_rating'].transform('std')
#combine_set['service_length_rating_nuq']=combine_set.groupby(['length_of_service'])['previous_year_rating'].transform('nunique')

In [ ]:
sns.boxplot(combine_set['age'])

In [16]:
combine_set['service_length_rating_sum']=combine_set.groupby(['length_of_service'])['previous_year_rating'].transform('sum')
#combine_set['service_length_rating_sd']=combine_set.groupby(['length_of_service'])['previous_year_rating'].transform('std')
#combine_set['service_length_rating_uni']=combine_set.groupby(['length_of_service'])['previous_year_rating'].transform('nunique')

In [17]:
combine_set['age_service_length_rating_sum']=combine_set.groupby(['length_of_service','age'])['previous_year_rating'].transform('sum')
#combine_set['age_service_length_rating_sd']=combine_set.groupby(['length_of_service','age'])['previous_year_rating'].transform('std')
#combine_set['age_service_length_rating_uni']=combine_set.groupby(['length_of_service','age'])['previous_year_rating'].transform('nunique')

In [18]:
combine_set['region_rating_sum']=combine_set.groupby(['region'])['previous_year_rating'].transform('sum')
#combine_set['region_rating_SD']=combine_set.groupby(['region'])['previous_year_rating'].transform('std')
#combine_set['region_rating_uni']=combine_set.groupby(['region'])['previous_year_rating'].transform('nunique')

In [19]:
combine_set['KPI_rating_sum']=combine_set.groupby(['KPIs_met >80%'])['previous_year_rating'].transform('sum')
#combine_set['KPI_rating_SD']=combine_set.groupby(['KPIs_met >80%'])['previous_year_rating'].transform('std')
#combine_set['KPI_rating_uni']=combine_set.groupby(['KPIs_met >80%'])['previous_year_rating'].transform('nunique')

In [ ]:
pd.set_option('display.max_columns', None)
combine_set.head()

In [ ]:
combine_set.shape

In [ ]:
combine_set['age_service_length_rating_uni'].value_counts()

In [20]:
from sklearn.preprocessing import KBinsDiscretizer

In [21]:
age_discretizer = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='quantile')

In [22]:
combine_set['Age_Bins'] =age_discretizer.fit_transform(combine_set['age'].values.reshape(-1,1)).astype(int)

In [23]:
train_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
service_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

In [24]:
combine_set['train_score_Bins'] =train_discretizer.fit_transform(combine_set['avg_training_score'].values.reshape(-1,1)).astype(int)
combine_set['service_Bins'] =service_discretizer.fit_transform(combine_set['length_of_service'].values.reshape(-1,1)).astype(int)

In [ ]:
combine_set.head()

In [25]:
region_discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
combine_set['region'] =region_discretizer.fit_transform(combine_set['region'].values.reshape(-1,1)).astype(int)

In [ ]:
combine_set.isnull().sum()

In [26]:
recruite_counts = combine_set['recruitment_channel'].value_counts().to_dict()

In [27]:
combine_set['recruitment_channel'] = combine_set['recruitment_channel'].map(recruite_counts)

In [28]:
combine_set.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,...,recruitment_rating_sum,education_rating_sum,awards_won_rating_sum,service_length_rating_sum,age_service_length_rating_sum,region_rating_sum,KPI_rating_sum,Age_Bins,train_score_Bins,service_Bins
0,65438,7,1,3,0,33181,1,35,5,8,...,109551,72036,252586,13834,1691,23042,106990,4,0,4
1,65141,4,3,2,1,43524,1,30,5,4,...,143228,183011,252586,32411,3121,30674,151944,2,2,2
2,7513,7,3,2,1,33181,1,34,3,7,...,109551,183011,252586,26354,2914,4270,151944,4,1,3
3,2542,7,3,2,1,43524,2,39,1,10,...,143228,183011,252586,10472,977,5701,151944,5,1,4
4,48945,8,4,2,1,43524,1,45,3,2,...,143228,183011,252586,31688,120,10701,151944,6,3,1


In [ ]:
combine_set['is_promoted'].value_counts()

In [ ]:
combine_set.head()

In [37]:
train_df=combine_set[combine_set['is_promoted'].isnull()==False]
test_df=combine_set[combine_set['is_promoted'].isnull()==True]

In [38]:
X=train_df.drop(['employee_id','is_promoted','age','region',
                'length_of_service','avg_training_score'],axis=1)
y=train_df['is_promoted']

X_main_test=test_df.drop(['employee_id','is_promoted','age','region',
                'length_of_service','avg_training_score'],axis=1)

In [39]:
#Kfold
kf=StratifiedKFold(n_splits=10,shuffle=True,random_state=123)
pred_score=0
preds=0

In [40]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54808 entries, 0 to 54807
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype
---  ------                         --------------  -----
 0   department                     54808 non-null  int32
 1   education                      54808 non-null  int64
 2   gender                         54808 non-null  int32
 3   recruitment_channel            54808 non-null  int64
 4   no_of_trainings                54808 non-null  int64
 5   previous_year_rating           54808 non-null  int32
 6   KPIs_met >80%                  54808 non-null  int64
 7   awards_won?                    54808 non-null  int64
 8   Total_Score                    54808 non-null  int64
 9   departmentwise_rating_sum      54808 non-null  int32
 10  recruitment_rating_sum         54808 non-null  int32
 11  education_rating_sum           54808 non-null  int32
 12  awards_won_rating_sum          54808 non-null  int32
 13  service_length_r

## Light GBM:

In [41]:
import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
gbm = lgb.LGBMClassifier()

In [42]:
for i, (train_idx,val_idx) in enumerate(kf.split(X,y)):
    X_train,y_train=X.iloc[train_idx,:],y.iloc[train_idx]
    X_val,y_val=X.iloc[val_idx,:],y.iloc[val_idx]
    
    print('\nFold: {}\n'.format(i+1))
    
    hyper_params = {
    "learning_rate":[0.05,0.1,0.15,0.2,0.25],
    "max_depth":[3,4,5,7,9,10,50],
    "min_data_in_leaf":[10,20,25,30],
    #"colsample_bytree":[0.3,0.4,0.5,0.6],
    "num_leaves " : [10,20,30,40],
    "eval_metric" : ['auc'],
    "num_leaves":[10,20,30,50,100],
    "boosting" : ['gbdt','goss'],
    #"n_estimators" : [100, 500, 900, 1100, 1500]
}
    
    random_search = RandomizedSearchCV(gbm,param_distributions = hyper_params,
                                  n_iter = 50,
                                  n_jobs = -1,cv = 5, verbose = 50)
    
    #lg=LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=10,
     #                 learning_rate=0.04,objective='binary',metric='auc',
      #                colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,
       #               random_state=294,n_jobs=-1)      
   

    random_search.fit(X_train,y_train)
    print(roc_auc_score(y_val,random_search.predict_proba(X_val)[:,1]))
    
    pred_score=roc_auc_score(y_val,random_search.predict_proba(X_val)[:,1])
    
    preds=random_search.predict_proba(X_main_test)[:,1]
    
print('mean_score: {}'.format(pred_score/10))

#preds_random_search=preds/10



Fold: 1

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    6.3s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    7.9s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | e

[Parallel(n_jobs=-1)]: Done 131 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 132 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.3min
[Paralle

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:   11.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:   12.9s
[Parallel(n_jobs=-1)]: Done  23 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done  31 tasks      | elapsed:   17.9s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   18.7s
[Paralle

[Parallel(n_jobs=-1)]: Done 151 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 153 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 155 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  1.3min
[Paralle

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:   18.9s
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:   19.3s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   21.2s
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   21.5s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   23.2s
[Parallel(n_jobs=-1)]: Done  47 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:   25.6s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:   27.1s
[Paralle

...

[Parallel(n_jobs=-1)]: Done 134 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 135 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 137 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 139 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 140 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 142 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 143 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 144 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 145 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 147 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 148 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  1.1min
[Paralle

[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done  22 tasks      | elapsed:   12.5s
[Parallel(n_jobs=-1)]: Done  23 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   13.0s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done  27 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   14.8s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:   16.1s
[Parallel(n_jobs=-1)]: Done  31 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   17.5s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   17.8s
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed:   18.4s
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:   18.7s
[Paralle

[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 155 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 156 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 157 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 159 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 161 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 162 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 163 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 165 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 166 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 167 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 169 tasks      | elapsed:  1.5min
[Paralle

[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   21.3s
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.3s
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:   23.6s
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   24.4s
[Parallel(n_jobs=-1)]: Done  47 tasks      | elapsed:   24.7s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done  49 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  51 tasks      | elapsed:   26.2s
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   27.3s
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:   27.6s
[Parallel(n_jobs=-1)]: Done  55 tasks      | elapsed:   27.7s
[Paralle

[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 174 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 175 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 177 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 178 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 179 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 180 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 181 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 182 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 183 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 185 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 186 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 187 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 188 tasks      | elapsed:  1.6min
[Paralle

In [45]:
preds

array([0.17814417, 0.0018906 , 0.0016861 , ..., 0.00180686, 0.00689614,
       0.26891741])

## XGBoost:

In [43]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost

In [44]:
clf = xgboost.XGBClassifier()

In [ ]:
for i, (train_idx,val_idx) in enumerate(kf.split(X,y)):
    X_train,y_train=X.iloc[train_idx,:],y.iloc[train_idx]
    X_val,y_val=X.iloc[val_idx,:],y.iloc[val_idx]
    
    print('\nFold: {}\n'.format(i+1))
    
    params = {
    "n_estimators" : [10, 50, 90,100,200,400],
    "max_depth" : [2, 3, 5, 10, 15,20,30],
    "booster": ['gbtree'],
    "learning_rate":[0.05,0.1,0.15,0.20,0.3],
    "min_child_weight":[1,2,3,4,5,10],
    "eval_metric": ['auc'],
    "gamma" : [0,0.01,0.02,0.1,0.2,0.3,0.05],
    #"scale_pos_weight ": [7] ## For imbalance data ([0]/[1]= 7 ,8,)
}
    
    # Set up the random search with 4-fold cross validation
    random_cv = RandomizedSearchCV(estimator=clf,
            param_distributions=params,
            cv=5,
            n_iter=10,
            n_jobs = -1,
            verbose = 50, 
            random_state=42,
            return_train_score = False) 
    #lg=LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=10,
     #                 learning_rate=0.04,objective='binary',metric='auc',
      #                colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,
       #               random_state=294,n_jobs=-1)      
   

    random_cv.fit(X_train,y_train)
    print(roc_auc_score(y_val,random_cv.predict_proba(X_val)[:,1]))
    #print(roc_auc_score(y_val,random_cv.predict(X_val)[:,1]))
    
    pred_score_XG = roc_auc_score(y_val,random_cv.predict_proba(X_val)[:,1])
    #pred_score_XG = roc_auc_score(y_val,random_cv.predict(X_val)[:,1])
    
    preds_XG = random_cv.predict_proba(X_main_test)[:,1]
    
print('mean_score: {}',np.mean(preds_XG))

#preds_random_search_XG=preds_XG/10



Fold: 1

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done  14 tasks      | el

[Parallel(n_jobs=-1)]: Done  32 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  7.5min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  8.0min
[Parallel(n_jobs=-1)]: Done  35 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:  8.4min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  8.8min
[Parallel(n_jobs=-1)]: Done  38 tasks      | elapsed:  8.9min
[Parallel(n_jobs=-1)]: Done  39 tasks      | elapsed:  9.0min
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  43 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  45 out of  50 | elapsed:  9.3min remaining:  1.0min
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:  9.3min remaining:   35.7s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  9.4min finished
0.895397811182223

Fold

In [ ]:
sample.head()

In [ ]:
#Submission File
sample['is_promoted1']=preds*0.3+preds_XG*0.7 #50%)
sample['is_promoted2']=preds*0.4+preds_XG*0.6
sample['is_promoted3']=preds*0.5+preds_XG*0.5
sample['is_promoted4']=preds*0.2+preds_XG*0.8

#sample.to_csv('C:/Users/Aditya/Desktop/Kaggle Datasets/Janata Hack Cross Sell/main_test.csv',index=False)
sample.head(15)


In [ ]:
sample['is_promoted5']=preds*0.33+preds_XG*0.77
sample['is_promoted6']=preds*0.25+preds_XG*0.75
    
#sample['is_promoted5']=np.where(sample['is_promoted5']>=1,1,0)
#sample['is_promoted6']=np.where(sample['is_promoted6']>=1,1,0)
    
    

In [ ]:
sample.head(15)

In [ ]:
sample.to_csv("C:/Users/Aditya/Desktop/HR Analytics/LGBM_XGB-bend2.csv",index=False)

In [ ]:
sample['is_promoted11']=np.where(sample['is_promoted1']>=0.8,1,0)
sample['is_promoted22']=np.where(sample['is_promoted2']>=0.7,1,0)
sample['is_promoted33']=np.where(sample['is_promoted3']>=6,1,0)
sample['is_promoted44']=np.where(sample['is_promoted4']>=65,1,0)


In [ ]:
sample['is_promoted33'].value_counts()
#sample['is_promoted2'].value_counts()
#sample['is_promoted3'].value_counts()
#sample['is_promoted4'].value_counts()
#sample['is_promoted4'].value_counts()
#sample['is_promoted5'].value_counts()
#sample['is_promoted6'].value_counts()

In [ ]:
sample.head(15)